In [1]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = float(v)
        self.weight = float(w)
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value)\
                 + ', ' + str(self.weight) + '>'
        return result

In [2]:
def value(item):
    return item.getValue()

In [3]:
def weightInverse(item):
    return 1.0/item.getWeight()

In [4]:
def density(item):
    return item.getValue() / item.getWeight()

In [5]:
def buildItems():
    names = ['時計', '絵画', 'ラジオ', '花瓶', '本', 'コンピュータ']
    values = [175, 90, 20, 50, 10, 200]
    weights = [10, 9, 4, 2, 1, 20]
    Items = []
    for i in range(len(values)):
        Items.append(Item(names[i], values[i], weights[i]))
    return Items

In [6]:
def greedy(items, maxWeight, keyFunction):
    """Itemsはリスト、maxWeight >= 0とし、
       keyFunctionはそれぞれのitemをその属性値にマップする"""
    itemsCopy = sorted(items, key=keyFunction, reverse = True)
    result = []
    totalValue = 0.0
    totalWeight = 0.0
    for i in range(len(itemsCopy)):
        if (totalWeight + itemsCopy[i].getWeight()) <= maxWeight:
            result.append(itemsCopy[i])
            totalWeight += itemsCopy[i].getWeight()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

In [7]:
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print '選択した項目の合計金額 =', val
    for item in taken:
        print '   ', item

In [8]:
def testGreedys(maxWeight = 20):
    items = buildItems()
    print 'サイズが' + str(maxWeight) + 'のナップザックに金額に対する貪欲アルゴリズムで詰める'
    testGreedy(items, maxWeight, value)
    print '\nサイズが' + str(maxWeight) + 'のナップザックに重量に対する貪欲アルゴリズムで詰める'
    testGreedy(items, maxWeight, weightInverse)
    print '\nサイズが' + str(maxWeight) + 'のナップザックに密度に対する貪欲アルゴリズムで詰める'
    testGreedy(items, maxWeight, density)

In [9]:
testGreedys()

サイズが20のナップザックに金額に対する貪欲アルゴリズムで詰める
選択した項目の合計金額 = 200.0
    <コンピュータ, 200.0, 20.0>

サイズが20のナップザックに重量に対する貪欲アルゴリズムで詰める
選択した項目の合計金額 = 170.0
    <本, 10.0, 1.0>
    <花瓶, 50.0, 2.0>
    <ラジオ, 20.0, 4.0>
    <絵画, 90.0, 9.0>

サイズが20のナップザックに密度に対する貪欲アルゴリズムで詰める
選択した項目の合計金額 = 255.0
    <花瓶, 50.0, 2.0>
    <時計, 175.0, 10.0>
    <本, 10.0, 1.0>
    <ラジオ, 20.0, 4.0>


In [10]:
def chooseBest(pset, maxWeight, getVal, getWeight):
    bestVal = 0.0
    bestSet = None
    for items in pset:
        itemsVal = 0.0
        itemsWeight = 0.0
        for item in items:
            itemsVal += getVal(item)
            itemsWeight += getWeight(item)
        if itemsWeight <= maxWeight and itemsVal > bestVal:
            bestVal = itemsVal
            bestSet = items
    return (bestSet, bestVal)

In [11]:
def getBinaryRep(n, numDigits):
    """nとnumDigitsを非負のint型とする
       nの値を，numDigits桁の2進数で表す文字列を返す"""
    result = ''
    while n > 0:
        result = str(n%2) + result
        n = n//2
    if len(result) > numDigits:
        raise ValueError('not enough digits')
    for i in range(numDigits - len(result)):
        result = '0' + result
    return result

In [12]:
def genPowerset(L):
    """Lをリストとする
       Lの要素の，すべての可能な組合せからなるリストを返す
       例えばLが[1, 2]ならば，
       [], [1], [2], [1,2] を要素にもつリストを返す"""
    powerset = []
    for i in range(0, 2 ** len(L)):
        binStr = getBinaryRep(i, len(L))
        subset = []
        for j in range(len(L)):
            if binStr[j] == '1':
                subset.append(L[j])
        powerset.append(subset)
    return powerset

In [13]:
def testBest(maxWeight = 20):
    items = buildItems()
    pset = genPowerset(items)
    taken, val = chooseBest(pset, maxWeight, Item.getValue,
                            Item.getWeight)
    print '選択した項目の合計金額 =', val
    for item in taken:
        print item

In [14]:
testBest()

選択した項目の合計金額 = 275.0
<時計, 175.0, 10.0>
<絵画, 90.0, 9.0>
<本, 10.0, 1.0>


In [15]:
class Node(object):
    def __init__(self, name):
        """name は文字列とする"""
        self.name = name
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

In [16]:
class Edge(object):
    def __init__(self, src, dest):
        """srcとdestは，どちらもNodeオブジェクトとする"""
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '->' + self.dest.getName()

In [17]:
class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        """srcとdestは，Nodeオブジェクトであるとし，
           weightはfloatとする"""
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return self.src.getName() + '->(' + str(self.weight) + ')'\
               + self.dest.getName()

In [18]:
class Digraph(object):
    #nodesはNodeオブジェクトのリストである
    #edgesは各nodeを，そのnodeのchildリストにマップする辞書である
    def __init__(self):
        self.nodes = []
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            raise ValueError('Duplicate node')
        else:
            self.nodes.append(node)
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.nodes
    def __str__(self):
        result = ''
        for src in self.nodes:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1] #最後の改行を省略する

In [19]:
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

In [20]:
def printPath(path):
    """path は Node オブジェクトからなるリストとする"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result

In [21]:
def DFS(graph, start, end, path, shortest):
    """graphはDigraphオブジェクト，startとendはNodeオブジェクト，
       pathとshortestは，Nodeオブジェクトのリストとする．
       graphでの，startノードからendノードへの最短路を返す"""
    
    path = path + [start]
    print '現在の深さ優先探索の経路:', printPath(path)
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #サイクルを避ける
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest)
                if newPath != None:
                    shortest = newPath
    return shortest

In [22]:
def search(graph, start, end):
    """graphはDigraphオブジェクト，startとendはNodeオブジェクトとする．
       graphでの，startノードからendノードへの最短路を返す"""
    return DFS(graph, start, end, [], None)

In [23]:
def testSP():
    nodes = []
    for name in range(6): #6つのノードを生成する
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[0]))
    g.addEdge(Edge(nodes[3],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[0]))
    sp = search(g, nodes[0], nodes[5])
    print '深さ優先探索で見つけた最短路:', printPath(sp)

In [24]:
testSP()

現在の深さ優先探索の経路: 0
現在の深さ優先探索の経路: 0->1
現在の深さ優先探索の経路: 0->1->2
現在の深さ優先探索の経路: 0->1->2->3
現在の深さ優先探索の経路: 0->1->2->3->4
現在の深さ優先探索の経路: 0->1->2->3->5
現在の深さ優先探索の経路: 0->1->2->4
現在の深さ優先探索の経路: 0->2
現在の深さ優先探索の経路: 0->2->3
現在の深さ優先探索の経路: 0->2->3->4
現在の深さ優先探索の経路: 0->2->3->5
現在の深さ優先探索の経路: 0->2->3->1
現在の深さ優先探索の経路: 0->2->4
深さ優先探索で見つけた最短路: 0->2->3->5


In [25]:
def BFS(graph, start, end):
    """graphはDigraphオブジェクト，startとendはNodeオブジェクトとする．
       graphでの，startノードからendノードへの最短路を返す"""
    initPath = [start]
    pathQueue = [initPath]
    while len(pathQueue) != 0:
        #pathQueueの中で一番古い要素を参照し，それを取り除く
        tmpPath = pathQueue.pop(0)
        print '現在の幅優先探索の経路:', printPath(tmpPath)
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None

In [26]:
def testSP():
    nodes = []
    for name in range(6): #6つのノードを生成する
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[0]))
    g.addEdge(Edge(nodes[3],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[0]))
    sp = search(g, nodes[0], nodes[5])
    print '深さ優先探索で見つけた最短路:', printPath(sp)
    sp = BFS(g, nodes[0], nodes[5])
    print '幅優先探索で見つけた最短路:', printPath(sp)

In [27]:
testSP()

現在の深さ優先探索の経路: 0
現在の深さ優先探索の経路: 0->1
現在の深さ優先探索の経路: 0->1->2
現在の深さ優先探索の経路: 0->1->2->3
現在の深さ優先探索の経路: 0->1->2->3->4
現在の深さ優先探索の経路: 0->1->2->3->5
現在の深さ優先探索の経路: 0->1->2->4
現在の深さ優先探索の経路: 0->2
現在の深さ優先探索の経路: 0->2->3
現在の深さ優先探索の経路: 0->2->3->4
現在の深さ優先探索の経路: 0->2->3->5
現在の深さ優先探索の経路: 0->2->3->1
現在の深さ優先探索の経路: 0->2->4
深さ優先探索で見つけた最短路: 0->2->3->5
現在の幅優先探索の経路: 0
現在の幅優先探索の経路: 0->1
現在の幅優先探索の経路: 0->2
現在の幅優先探索の経路: 0->1->2
現在の幅優先探索の経路: 0->2->3
現在の幅優先探索の経路: 0->2->4
現在の幅優先探索の経路: 0->1->2->3
現在の幅優先探索の経路: 0->1->2->4
現在の幅優先探索の経路: 0->2->3->4
現在の幅優先探索の経路: 0->2->3->5
幅優先探索で見つけた最短路: 0->2->3->5
